In [15]:
# change these to try this notebook out
BUCKET = 'opioid-care'
PROJECT = 'opioid-care'
REGION = 'us-central1'

In [16]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [17]:
import pandas as pd
import time
import re

pd.options.display.max_columns=35

<h2> Using the model to predict </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict death rate

In [18]:
test_data = pd.read_json('data/test-data.json')
test_data

,age_adjusted_mortality,income_20th_percentile,injury_death_rate,length_of_life_pctile_within_state,length_of_life_quartile_within_state,median_household_income,mentally_unhealthy_days,pct_children_eligible_free_lunch,pct_children_in_poverty,pct_diabetic,pct_excessive_drinking,pct_fair_or_poor_health,pct_frequent_mental_distress,pct_frequent_physical_distress,pct_insufficient_sleep,pct_physically_inactive,pct_smokers,pct_some_college,pct_unemployed,physically_unhealthy_days,preventable_hospitalization_rate,state
0,239.8,31238,34.208234,3,1,72131,3.3,40.766148,15.6,9.3,21.2,15.7,9.7,10.2,32.7,18.9,14.2,59.309657,6.959381,3.5,57.68,IL
1,313.8,27176,64.114307,39,2,56243,2.8,30.445732,14.9,9.3,19.8,11.3,8.4,8.4,26.3,26.9,14.7,67.886179,3.681616,2.8,47.25,IA
2,364.4,17433,80.752423,28,2,41595,3.5,41.060606,20.2,10.7,18.7,14.5,10.8,11.9,28.2,31.8,18.4,53.806452,5.362463,3.9,81.13,MT
3,541.7,17998,74.737758,66,3,39546,4.6,54.964029,26.8,12.5,13.8,22.0,14.3,14.8,40.6,33.2,21.0,48.927875,7.137850,4.8,73.81,AL
4,533.2,16072,79.536699,99,4,40122,4.3,56.222847,23.4,12.2,14.4,19.4,13.1,12.6,37.3,34.8,22.9,44.134768,7.989917,4.1,63.71,IN


In [31]:
request_data_json = [{"state":"MO",
                      "mentally_unhealthy_days":44.7,
                      "physically_unhealthy_days":4.0,
                      "pct_frequent_mental_distress":11.6,
                      "pct_excessive_drinking":9.0,
                      "injury_death_rate":52.88948031,
                      "age_adjusted_mortality":304.2,
                      "pct_frequent_physical_distress":12.0,
                      "pct_smokers":19.5,
                      "pct_insufficient_sleep":29.3,
                      "preventable_hospitalization_rate":41.4,
                      "pct_some_college":40.7502029,
                      "pct_fair_or_poor_health":15.8,
                      "pct_unemployed":8.12557695,
                      "length_of_life_pctile_within_state":7.0,
                      "median_household_income":50305.0,
                      "pct_physically_inactive":19.5,
                      "length_of_life_quartile_within_state":1.0,
                      "pct_children_in_poverty":17.7,
                      "income_20th_percentile":17667.0,
                      "pct_children_eligible_free_lunch":30.74736162,
                      "pct_diabetic":7.9}
                ]

In [32]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

PROJECT = 'opioid-care'  # Change to GCP project where the Cloud ML Engine model is deployed
MODEL_NAME = 'drugdeaths'  # Change to the deployed Cloud ML Engine model
MODEL_VERSION = "v1"  # If None, the default version will be used

def predict(instances):
    """ Use a deployed model to Cloud ML Engine to perform prediction

    Args:
        instances: list of json, csv, or tf.example objects, based on the serving function called
    Returns:
        response - dictionary. If no error, response will include an item with 'predictions' key
    """

    credentials = GoogleCredentials.get_application_default()

    service = discovery.build('ml', 'v1', credentials=credentials)
    model_url = 'projects/{}/models/{}'.format(PROJECT, MODEL_NAME)

    if MODEL_VERSION is not None:
        model_url += '/versions/{}'.format(MODEL_VERSION)

    request_data = {
        'instances': instances
    }

    response = service.projects().predict(
        body=request_data,
        name=model_url
    ).execute()

    output = response.values()
    print(request_data,"\n")
    return output

In [33]:
predict(request_data_json)

{'instances': [{'pct_children_eligible_free_lunch': 30.74736162, 'pct_unemployed': 8.12557695, 'median_household_income': 50305.0, 'pct_insufficient_sleep': 29.3, 'income_20th_percentile': 17667.0, 'injury_death_rate': 52.88948031, 'pct_smokers': 19.5, 'preventable_hospitalization_rate': 41.4, 'pct_some_college': 40.7502029, 'pct_fair_or_poor_health': 15.8, 'pct_excessive_drinking': 9.0, 'pct_children_in_poverty': 17.7, 'pct_frequent_physical_distress': 12.0, 'length_of_life_pctile_within_state': 7.0, 'pct_physically_inactive': 19.5, 'state': 'MO', 'length_of_life_quartile_within_state': 1.0, 'mentally_unhealthy_days': 44.7, 'pct_diabetic': 7.9, 'physically_unhealthy_days': 4.0, 'age_adjusted_mortality': 304.2, 'pct_frequent_mental_distress': 11.6}]} 



dict_values([[{'predictions': [1.6489917039871216]}]])